In [303]:
import os

# Specify the file path
file_path = "TC_database.db"

# Check if the file exists before attempting to delete it
if os.path.exists(file_path):
    # Delete the file
    os.remove(file_path)
    print(f"{file_path} deleted successfully")
else:
    print(f"{file_path} does not exist")


TC_database.db deleted successfully


In [304]:
import sqlite3

conn = sqlite3.connect('TC_database.db')

c = conn.cursor()

In [305]:
# Execute the PRAGMA foreign_keys command
c.execute("PRAGMA foreign_keys = ON")
c.execute("PRAGMA foreign_keys;")

# Fetch and print the result
result = c.fetchone()[0]
print("Foreign Keys Enabled:", result)

Foreign Keys Enabled: 1


In [306]:
# c.execute("""create table FunctionModule(
#           module_id integer primary key,
#           module text unique
# )""")

# conn.commit()

In [307]:
c.execute("""create table TriggeringCondition(
          TC_id integer primary key,
          TC_name text not null unique,
          FI text not null,
          FI_origin not null
)""")

conn.commit()

In [308]:
# # functional_insufficiency text not null CHECK (functional_insufficiency IN ('Localization','Perception','Prediction','Planning','Control')),
# c.execute("""create table FunctionalInsufficiency(
#           FI_id integer primary key,
#           TC_id integer not null,
#           functional_insufficiency text not null CHECK (functional_insufficiency IN ('Localization','Perception','Prediction','Planning','Control')),
#           origin text not null,
#           foreign key (TC_id) references TriggeringCondition(TC_id),
#           foreign key (functional_insufficiency) references FunctionModule(module),
#           UNIQUE(TC_id,functional_insufficiency,origin)
# )""")

# conn.commit()

In [309]:
c.execute("""create table DrivingBehaviorRequirement(
          behavior_requirement_id integer primary key,
          behavior_requirement text not null
)
""")

conn.commit()

In [310]:
c.execute("""create table TCEnhancedAbstractScenario(
          abstract_scenairo_id integer primary key,
          TC_id integer not null,
          abstract_scenario text not null,
          exist_non_complient integer not null,
          foreign key(TC_id) references TriggeringCondition(TC_id)
)
""")

conn.commit()

In [311]:
c.execute("""create table AbstractScenarioRequirement(
          abstract_scenairo_id integer not null,
          behavior_requirement_id integer not null,
          foreign key(abstract_scenairo_id) references TCEnhancedAbstractScenario(abstract_scenairo_id)
          foreign key(behavior_requirement_id) references DrivingBehaviorRequirement(behavior_requirement_id)
)
""")

In [312]:
c.execute("""create table TCEnhancedConcreteScenario(
          concrete_scenairo_id integer primary key,
          abstract_scenario_id integer not null,
          concrete_scenario text not null,
          foreign key(abstract_scenario_id) references TCEnhancedAbstractScenario(abstract_scenario_id)
)
""")

conn.commit()

In [313]:
c.execute("""create table ConcreteScenarioRequirement(
          concrete_scenairo_id integer not null,
          behavior_requirement_id integer not null,
          foreign key(concrete_scenairo_id) references TCEnhancedConcreteScenario(concrete_scenairo_id)
          foreign key(behavior_requirement_id) references DrivingBehaviorRequirement(behavior_requirement_id)
)
""")

In [314]:
c.execute("""create table HazardousBehavior(
          hazardous_behavior_id integer primary key,
          hazardous_behavior text not null
)
""")

conn.commit()

In [315]:
c.execute("""create table HazardousEvent(
          hazardous_event_id integer primary key,
          hazardous_event text not null
)
""")

conn.commit()

In [316]:
c.execute("""create table ConcreteScenarioTestingResult(
          result_id integer primary key,
          TC_id integer not null,
          concrete_scenairo_id integer,
          system_info text not null,
          test_result text not null,
          hazardous_behavior_id integer,
          hazardous_event_id integer,
          testing_date text not null,
          log_file text not null,
          foreign key(TC_id) references triggeringConditions(TC_id),
          foreign key(concrete_scenairo_id) references TCEnhancedConcreteScenario(concrete_scenairo_id),
          foreign key(hazardous_behavior_id) references HazardousBehavior(hazardous_behavior_id),
          foreign key(hazardous_event_id) references HazardousEvent(hazardous_event_id)
)
""")

conn.commit()

In [317]:
conn.commit()
conn.close()

Function Modules

In [318]:
import os

conn = sqlite3.connect('TC_database.db')

c = conn.cursor()

In [319]:
# modules = ['Localization','Perception','Prediction','Planning','Control']

# for module in modules:
#     c.execute("insert into FunctionModule values(NULL,?)", (module,))

# conn.commit()

TCs

In [320]:
from sqlite3 import IntegrityError
TCs = [
    {'TC':'Parking vehicle on the roadside with opening door', 'FI':'Information processing-lidar-based object classification, Plan-behavior prediction'},
    {'TC':'E-scooter drives on motor lane', 'FI':'Plan-situation understanding'},
    {'TC':'Vehicle from the right merging lane without right of way brakes too late', 'FI':'Plan-prediction'},
    {'TC':'Workers unload a truck on the road', 'FI':'Information processing-object detection, Plan-trajectory prediction'},
    {'TC':'Pedestrian halts sharply in front of the crosswalk', 'FI':'Plan-trajectory prediction'},
    {'TC':'Front cyclist leaves the lane', 'FI':'Information processing - object detection, Plan - situation understanding'},
    {'TC':'Multiple vehicles cut in to ego path in a short time', 'FI':'Plan - situation understanding & decision making'},
    {'TC':'Front vehicle drives on lane markings', 'FI':'Plan-situation understanding'},
    {'TC':'Many vehicles park along the road side', 'FI':'Information access - offline map backup, Information processing - localization, road mark detection, Plan-situation understanding, prediction'},
    {'TC':'Traffic jam in front when neighbor lane is free', 'FI':'Plan - situation understanding + prediction'},
    {'TC':'Front vehicle halts on the lane with warning flasher on', 'FI':'Plan - situation understanding'},
    {'TC':'Cyclist crosses from the roadside', 'FI':'Information reception - FOV,, Plan - Prediction'},
    {'TC':'Oncoming vehicle overtakes a halting vehicle', 'FI':'Plan-situation understanding'},
    {'TC':'Scooter/cyclist drives on opposite lane', 'FI':'Plan-situation understanding'},
    {'TC':'Traffic jam on the opposite lane', 'FI':'Information processing-object separation, Plan-situation understanding, prediction'},
    {'TC':'Halting vehicles on both directions narrow the road', 'FI':'Plan-situation understanding'},
    {'TC':'Front cyclist moves towards the road edge and halt', 'FI':'Plan-behavior interpretation & prediction'},
]


for TC in TCs:
    try:
        c.execute("insert into TriggeringCondition values(NULL,?,?,?)", (TC['TC'],TC['FI'],'Expert'))
    except IntegrityError:
        pass
conn.commit()

In [321]:
abstract_scenarios = [
    {'TC_id':6,'abstract_scenario':'Ego follows lane, front cyclist leaves the lane to the bus station on the right','exist_non_complient':0},
    {'TC_id':6,'abstract_scenario':'Ego change lane to right, front cyclist also changes to the right','exist_non_complient':0},
    {'TC_id':13,'abstract_scenario':"The road has one lane for ego's travelling direction and one lane for the opposite direction. On the opposite lane a delivery vehicle halts with warning flasher. The vehicle behind it performs overtaking via ego's lane.",'exist_non_complient':1},
    {'TC_id':17,'abstract_scenario':'Front cyclist moves towards the road edge to talk with a pedestrian','exist_non_complient':0},
]


for abstract_scenario in abstract_scenarios:
    c.execute("insert into TCEnhancedAbstractScenario values(NULL,?,?,?)", (abstract_scenario['TC_id'],abstract_scenario['abstract_scenario'],abstract_scenario['exist_non_complient']))

conn.commit()

In [322]:
home_dir = os.path.expanduser("~")
scenic_project_dir = os.path.join(home_dir, 'Tools/Scenic/scenic_projects/')
concrete_scenarios = [
    {'abs_id':1,'path':'17_front_cyclist_leaves_the_lane_1/17_front_cyclist_leaves_the_lane_1.scenic'},
    {'abs_id':2,'path':'17_front_cyclist_leaves_the_lane_2/17_front_cyclist_leaves_the_lane_2.scenic'},
    {'abs_id':3,'path':'Zhijing_scenario/Zhijing_scenario.scenic'},
    {'abs_id':4,'path':'21_Front_cyclist_moves_towards_the_road_edge_and_halt_1/21_Front_cyclist_moves_towards_the_road_edge_and_halt_1.scenic'},
]


for concrete_scenario in concrete_scenarios:
    c.execute("insert into TCEnhancedConcreteScenario values(NULL,?,?)", (concrete_scenario['abs_id'],concrete_scenario['path']))

conn.commit()